#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 2:  Link Analysis -- HITS + SEO

### 100 points [5% of your final grade]

### Due: Sunday, February 25, 2018 by 11:59pm

*Goals of this homework:* Explore real-world challenges of building a graph (in this case, from tweets), implement and test HITS algortihm over this graph, and investigate factors that impact a page's rank on Google and Bing.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw2.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Wednesday, February 28, 2018 at 11:59pm.

# Part 1: HITS (70 points)

## A re-Tweet Graph

In this assignment, we're going to adapt the classic HITS approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their retweets of other Twitter users (so user = node, retweet of another user = edge). Over this Twitter-user graph, we can apply the HITS approach to order the users by their hub-ness and their authority-ness.

Here is a toy example. Suppose you are given the following four retweets:

* **userID**: diane, **text**: "RT ", **sourceID**: bob
* **userID**: charlie, **text**: "RT Welcome", **sourceID**: alice
* **userID**: bob, **text**: "RT Hi ", **sourceID**: diane
* **userID**: alice, **text**: "RT Howdy!", **sourceID**: parisa

There are four short tweets retweeted by four users. The retweet between users form a directed graph with five nodes and four edges. E.g., the "diane" node has a directed edge to the "bob" node.

You should build a graph by parsing the tweets in the file we provide called *HITS.json*.

**Notes:**

* You may see some weird characters in the content of tweets, just ignore them. 
* The edges are weighted and directed. If Bob retweets Alice's tweets 10 times, there is an edge from Bob to Alice with weight 10, but there is not an edge from Alice to Bob.
* If a user retweets herself, ignore it.
* Correctly parsing screen_name in a tweet is error-prone. Use the id of the user (this is the user who is re-tweeting) and the id of the user in the retweeted_status field (this is the user who is being re-tweeted; that is, this user created the original tweet).
* Later you will need to implement the HITS algorithm on the graph you build here.


In [24]:
import urllib2
import json
import collections
import numpy as np
from urllib2 import urlopen
from __future__ import print_function

url = "https://piazza.com/class_profile/get_resource/jc7ucudxx0r11/jdq56nwvvabp4"

response = urlopen(url)
data = response.read()
final_data = [s.strip() for s in data.splitlines()]

node_rank = collections.defaultdict()
hubs = set()
graph = np.zeros((1003,1003))
h_dict = {}
a_dict = {}
h_list = []
a_list = []

for x in final_data:
    user = json.loads(x)
    userId = user['user']['id']
    sourceId = user['retweeted_status']['user']['id']
    if userId != sourceId:
        hubs.add(userId)
        hubs.add(sourceId)

i=0
for hub in hubs:
    node_rank[hub] = i
    h_dict[hub] = 0
    a_dict[hub] = 0
    h_list.append(tuple((hub, 0)))
    a_list.append(tuple((hub, 0)))
    i+=1
    
for x in final_data:
    user = json.loads(x)
    userId = user['user']['id']
    sourceId = user['retweeted_status']['user']['id']
    if userId != sourceId:
        graph[node_rank[userId]][node_rank[sourceId]] +=1

We will not check the correctness of your graph. However, this will affect the HITS results later.

## HITS Implementation

Your program will return the top 10 users with highest hub and authority scores. The **output** should be like:

Hub Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

Authority Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal scores.
* It is up to you to decide when to terminate the HITS calculation.
* There are HITS implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions.
* Test your parsing and HITS calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

In [32]:
# your code here
import operator
s = (1003,1)
h = np.ones(s)
for i in range(10000):
    a = np.dot(np.transpose(graph),h)
    h = np.dot(graph,a)
    a = a/np.sum(a)
    h = h/np.sum(h)

for i in range(len(h)):
    h_dict[h_list[i][0]] = float(h[i])
    a_dict[a_list[i][0]] = float(a[i])

sorted_h = sorted(h_dict.items(), key=operator.itemgetter(1),reverse=True)
sorted_a = sorted(a_dict.items(), key=operator.itemgetter(1),reverse=True)


In [36]:
print("Top 10 Hub scores:")
for i in range(10):
    print("#",i+1,sorted_h[i])

Top 10 Hub scores:
# 1 (3068706044, 0.06461002796552665)
# 2 (3093940760, 0.030708356925328948)
# 3 (2194518394, 0.026945710535333872)
# 4 (2862783698, 0.021005735950331943)
# 5 (3092183276, 0.017680745885625796)
# 6 (3029724797, 0.017314871502541268)
# 7 (2990704188, 0.015318195195171498)
# 8 (3001500121, 0.015016796720630781)
# 9 (3086921438, 0.013391401117772157)
# 10 (3042686360, 0.012988330068305246)


In [37]:
print("Top 10 Authority scores:")
for i in range(10):
    print("#",i+1,sorted_a[i])

Top 10 Authority scores:
# 1 (3042570996, 0.1070915940976824)
# 2 (3065514742, 0.09698300819613175)
# 3 (1638625987, 0.08729174026106754)
# 4 (3077733683, 0.05635351120367857)
# 5 (3039321886, 0.04411153844791682)
# 6 (3077695572, 0.02396653361993896)
# 7 (3019659587, 0.02226440559921848)
# 8 (1358345766, 0.01927630140445489)
# 9 (3061155846, 0.018480679440477304)
# 10 (3092580049, 0.018420500477813406)


# Part 2: Search Engine Optimization (30 + 5 points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **kbeznak parmatonic** --- two terms, lower case, no quote. As of today (Feb 16, 2018), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization#White_hat_versus_black_hat_techniques).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you (e.g., your NetID, but not the UIN!).
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 20 points for a well-reasoned discussion of your strategy
* 5 points for your page appearing in the search results by Google or Bing (no matter how is the ranking)

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?

http://people.tamu.edu/~yashvardhan/kbeznakparmatonic.html

What's your strategy? (2-4 paragraphs)


As we know from the PageRank algorithm, the more inlinks we are able to create for our page, the better it will rank on google search. So, I created my page on http://people.tamu.edu/~yashvardhan on tamu.edu and created other webpages using github webpages. Now I tried to link the other pages that I created to my webpage and tried to add the keyword "kbeznak parmatonic" in my webpage a number of times to improve its term-frequency, meanwhile also having a meaningful sentence creation. Also I added the keyword in the title which I think provides a good visibility on google.

I created a post on Craigslist and tried to link my webpage to it. This helped me increase my visibility on google. Also, I have created a google maps page (submitting my website as a business) to link to my website. But it is still to be approved by google. I created a post on "kbeznak parmatonic" on the personal blog (https://yashvardhan90.github.io/2018-02-25-kbeznak-parmatonic/). The blog linked to my webpage and helped increased its inlinks, hence the pagerank. I have also asked my fellow classmates to link to my webpage and asked a lot of my friends to visit my webpage. 

I also looked for latest search trends on google and included those terms in my website. This will help me get visibility to a lot of users and thus help me increase my pagerank.